# Assignment 2

Instructions: Implement both PG and an evolutionary algorithm to solve the Open AI Gym Lunar Lander problem, and then apply it to my area of choice, which is chess.

First, we need to do some setup

In [ ]:
import torch
import numpy as np
import gym

# Set the device
if torch.cuda.is_available():
    device = "gpu" # 🧮
# elif torch.backends.mps.is_available():
#     device = "mps" # 🧠
else:
    device = "cpu" # 🥺
    
print(f"Using device: {device}")

First, we need to write the code for our Policy Gradient function with a baseline (taken from REINFORCE). I'm going to use PyTorch as my neural network library (I want to try JAX, but this is the more practical choice for me at the moment. Exploration-Exploitation tradeoff 🤷‍♂️).

I'm going to start with a basic feed forward net for both the network that chooses the policy and the network that learns states' values.

First, the policy network for choosing actions

In [ ]:
from torch import nn

class PolicyChoice(nn.Module):
    def __init__(self):
        super(PolicyChoice, self).__init__()
        self.flatten = nn.Flatten()
        self.policy = nn.Sequential(
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 4) # logits; PyTorch built-in loss applies softmax later
        )

    def forward(self, x):
        probs = self.policy(x)
        return probs

policy_model = PolicyChoice().to(device)
policy_adam = torch.optim.Adam(policy_model.parameters(), 1e-3)

For our loss function for the policy network, we want to adjust just the parameters with the primary aim of affecting the probability of taking the action that we took on that time step. If the return of the resulting state is better than expected, we want to increase it proportionally. If it is less than expected, we want to decrease it proportionally. Thus, we multiply the gradient of the parameter weights w.r.t. the taken action's probability by the difference of the return for that state-action pair.

Importantly, there is an extra factor however that we must consider; when we decide that we want to take the gradient of the parameters w.r.t. a specific action's return, the policy expectancy must be multiplied by the specific action's likelihood to determine the value it contributes to the policy. Thus, we end up with the gradient of the action's probability conditioned on the state and parameters. 

Thus, the general concept of loss to backpropogate in the REINFORCE algorithm is:


$\Large (G_t - \hat{\upsilon}) \frac{\nabla\pi(A_t|S_t, \theta)}{\pi(A_t|S_t, \theta)}$

This can be expressed as:

$\Large (G_t - \hat{\upsilon}) \nabla \ln{\pi(A_t|S_t, \theta)}$


The code below just worries about the loss and not the gradient, as PyTorch provides autograd differentiation behind the scenes.

In [ ]:
def policy_loss(logits, state_util_difference, action_idx):
    ce_loss = nn.CrossEntropyLoss()
    actions_tensor = torch.zeros(4)
    actions_tensor[action_idx] = 1
    return ce_loss(logits, actions_tensor) * state_util_difference

Now, the network for approximating state utililities.

In [ ]:
class StateUtility(nn.Module):
    def __init__(self):
        super(StateUtility, self).__init__()
        self.flatten = nn.Flatten()
        self.state_utility = nn.Sequential(
            # nn.Linear(8, 4),
            # nn.ReLU(),
            # nn.Linear(4, 2), # TODO: try reducing to one hidden layer if learning proves initially dificult
            # nn.ReLU(),
            # nn.Linear(2, 1), # output a tensor of a scalar value
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1), # output a tensor of a scalar value
        )

    def forward(self, x):
        state_utility = self.state_utility(x)
        return state_utility

state_util_model = StateUtility().to(device)
state_util_adam = torch.optim.Adam(state_util_model.parameters(), 1e-2)

For the state utilities network, we just use L1 loss with the gradients of W with respect to state utility.

$\Large (G_t - \hat{\upsilon}(S_t, W)) \nabla \hat{\upsilon}(S_t, W)$

Like above, the code below just worries about the loss and not the gradient, as PyTorch provides autograd differntiation.

In [ ]:
def state_util_loss(calculated_state_value, episode_state_value):
    # the overall state value is the input, and the individual state value is our target
    l1_loss = nn.L1Loss()
    return l1_loss(calculated_state_value, episode_state_value)


Let's define our hyperparameters

Having limitied compute (and longer runtimes) led to me to reflect on gamma tuning. I initially noticed that having a higher discount factor (smaller gamma) improved values a lot for this Lunar Lander task. My first thought was that it's a reflection of the fact that this task has very well-defined rewards that are frequent and that reflect short-term actions.

That's definitely true, but as I reflected on it, it also occurred to me that using a higher discount factor is, in general, a tradeoff. With a higher discount factor you are hamstringing your ability to learn long-term dependencies, but you can learn action's values much faster. However with a very low discount factor you can actually still theoretically learn actions fine grained values and not conflate them, but it just takes a lot more training examples as the Monte Carlo nature of sampling will eventually lead to distinction (my models with higher gammas seemed to still be learning when they terminated). And with this lower factor you can also learn long-term dependencies fairly easy.

I interpreted this as meaning that I should leave gamma a little higher than it's best value.

In [ ]:
gamma = .4

# gamma = .2

Let's load the Lunar Lander environment now

In [ ]:
# torch.autograd.set_detect_anomaly(mode=True)

In [ ]:
# TODO: use a custom dataloader class and see if speed up

env = gym.make(
    "LunarLander-v2",
    #render_mode="human"
)

action_space_seed = np.random.seed(13)

observation, info = env.reset(seed=13)

episodes_total_rewards = []
# for debug of state-value funtion
episode_total_state_err = []

# index i in the lists below corresponds to the timestep i of the current episode
observations = []
rewards = []
action_indices = []
action_logits_per_ep = []
# for debug of state-value funtion
state_err = []

policy_adam.zero_grad()
state_util_adam.zero_grad()


#for timestep in range(1000000):
for timestep in range(100000):

    if timestep==0 or timestep==30000:
        print('debug entry')
    
    # use policy gradient to get action probabilities; sample stochastically
    action_logits = policy_model(torch.tensor(observation, device=device, dtype=torch.float32))
    with torch.no_grad():
        action_logits_per_ep.append(action_logits.detach().clone())
        action_probs = np.array(torch.nn.functional.softmax(action_logits, dim=0)).tolist()
        action_sampling = np.random.multinomial(n=1, pvals=action_probs)
        action = np.argmax(action_sampling)
        action_indices.append(action)
    
    # get info from environment
    observation, reward, terminated, truncated, info = env.step(action)
    observations.append(observation)
    rewards.append(reward)
    
    # end of episode
    if terminated or truncated:
        ep_length = len(observations)
        ep_total_reward = np.sum(np.array(rewards))
        episodes_total_rewards.append(ep_total_reward)
        returns = np.zeros(len(observations))
        for timestep in reversed(range(ep_length)):

            # calculate state's actual return by looking at reward + future rewards
            terminal = timestep == len(rewards) - 1
            returns[timestep] = rewards[timestep] + (gamma * returns[timestep+1]) if not terminal else rewards[timestep]
            # calculate baseline expected state value
            pred_state_util = state_util_model(torch.tensor(observations[timestep], device=device, dtype=torch.float32))
            actual_state_util = torch.tensor([returns[timestep]], device=device, dtype=torch.float32)
            loss_state_utility = state_util_loss(pred_state_util, actual_state_util)
            with torch.no_grad():
                state_pred_err = np.abs(loss_state_utility.item())
                state_err.append(state_pred_err)

            # update state utility function
            loss_state_utility.backward()

            # make updates to policy (specific action) based on return
            with torch.no_grad():
                # get the state's return minus the basline (predicted state return)
                state_util_difference = actual_state_util - pred_state_util
            # loss_policy = policy_loss(action_logits_per_ep[timestep], state_util_difference, action_indices[timestep])
            # TODO: change to store rather than recomputation, but without autograd complaining about inplace operations
            loss_policy = policy_loss(policy_model(torch.tensor(observation, device=device, dtype=torch.float32)), state_util_difference, action_indices[timestep])

            # update policy gradients
            loss_policy.backward()

        episode_total_state_err.append(np.sum(np.array(state_err)))

        # sum gradients for state value parameters
        state_util_adam.step()
        state_util_adam.zero_grad()

        # perform policy network update
        policy_adam.step()
        policy_adam.zero_grad()

        observation, info = env.reset()
        observations, rewards, action_indices, action_logits_per_ep = [], [], [], []
        state_err = []

# TODO: move these to a self-contained function

print(f'The avg. state val prediction error on the first quarter of episodes was: {np.sum(episode_total_state_err[:len(episode_total_state_err)//4]) / len(episode_total_state_err)//4}')
print(f'The avg. state val prediction error on the second quarter of episodes was: {np.sum(episode_total_state_err[len(episode_total_state_err)//4:2 * len(episode_total_state_err)//4]) / len(episode_total_state_err)//4}')
print(f'The avg. state val prediction error on the third quarter of episodes was: {np.sum(episode_total_state_err[2 * len(episode_total_state_err)//4:3 *len(episode_total_state_err)//4]) / len(episode_total_state_err)//4}')
print(f'The avg. state val prediction error on the fourth quarter of episodes was: {np.sum(episode_total_state_err[3 *len(episode_total_state_err)//4:len(episode_total_state_err)]) / len(episode_total_state_err)//4}')

print(f'The avg. episode reward on the first quarter of episodes was: {np.sum(episodes_total_rewards[:len(episodes_total_rewards)//4]) / len(episodes_total_rewards)//4}')
print(f'The avg. episode reward on the second quarter of episodes was: {np.sum(episodes_total_rewards[len(episodes_total_rewards)//4:2 * len(episodes_total_rewards)//4]) / len(episodes_total_rewards)//4}')
print(f'The avg. episode reward on the third quarter of episodes was: {np.sum(episodes_total_rewards[2 * len(episodes_total_rewards)//4:3 *len(episodes_total_rewards)//4]) / len(episodes_total_rewards)//4}')
print(f'The avg. episode reward on the fourth quarter of episodes was: {np.sum(episodes_total_rewards[3 *len(episodes_total_rewards)//4:len(episodes_total_rewards)]) / len(episodes_total_rewards)//4}')
env.close()

In [ ]:
print(torch.__version__)
print(gym.__version__)